In [ ]:
import render_optimized as render #################à
import rooms.dataset
import evaluate
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import metrics

import timeit

In [ ]:
torch.set_default_dtype(torch.float32)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
D_0 = rooms.dataset.dataLoader("classroomBase")

In [ ]:
L_0 = render.get_listener(source_xyz=D_0.speaker_xyz, listener_xyz = D_0.xyzs[0], surfaces=D_0.all_surfaces, 
                                                speed_of_sound=D_0.speed_of_sound,
                                                parallel_surface_pairs=D_0.parallel_surface_pairs,
                                                max_order=D_0.max_order, max_axial_order=D_0.max_axial_order)

In [ ]:
R_0 = render.Renderer(n_surfaces=len(D_0.all_surfaces))

In [ ]:
# Use many GPUs if available
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs")
    R_0 = nn.DataParallel(R_0).module

In [ ]:
pt_file = torch.load('./models/classroomBase/weights.pt', map_location=device)
R_0.energy_vector = nn.Parameter(pt_file['model_state_dict']['energy_vector'])
R_0.source_response = nn.Parameter(pt_file['model_state_dict']['source_response'])
R_0.directivity_sphere = nn.Parameter(pt_file['model_state_dict']['directivity_sphere'])
R_0.decay = nn.Parameter(pt_file['model_state_dict']['decay'])
R_0.RIR_residual = nn.Parameter(pt_file['model_state_dict']['RIR_residual'])
R_0.spline_values = nn.Parameter(pt_file['model_state_dict']['spline_values'])

R_0.bp_ord_cut_freqs.to(device) 

In [ ]:
R_0.mic_180_loss = {1000: 0, 2000:0.05, 4000: 0.12, 6000:0.15, 16000:0.3, 20000:0.5 }

R_0.mic_0_gain = {1000: 1, 2000: 1, 4000: 0.9, 6000: 0.87, 16000: 1.07, 20000: 1.48}
#R_0.mic_180_loss = {1000: 0, 2000:0, 4000: 0, 6000:0, 16000:0, 20000:0 }
R_0.mic_direction = torch.Tensor([0,1,0])#####################

a = timeit.timeit(lambda:R_0.render_early_microphone_response(L_0), number=5)


print ("Elapsed:", a)

In [ ]:
R_0.cardioid_exponents = {1000:0.3, 5000:0.4, 10000:0.4, 16000:1.2}

R_0.mic_0_gain = {1000: 1,  5000: 1, 10000: 1, 16000: 1}
#R_0.mic_180_loss = {1000: 0, 2000:0, 4000: 0, 6000:0, 16000:0, 20000:0 }
R_0.mic_direction = torch.Tensor([0,1,0])#####################

a = timeit.timeit(lambda:R_0.render_early_cardioid(L_0), number=5)



print ("Elapsed:", a)

In [ ]:
a = timeit.timeit(lambda:R_0.render_early(L_0), number=5)




print ("Elapsed:", a)

In [ ]:
import fibonacci_utilities as fib
azimuths, elevations = fib.fibonacci_azimuths_and_elevations(3)

In [ ]:
a = timeit.timeit(lambda:R_0.render_early_directional(L_0, azimuths, elevations), number=5)


print ("Elapsed:", a)
